In [1]:
import pandas as pd

# 원본 데이터 (전처리 전)
df_raw = pd.read_csv("new_flight_data.csv")

numeric_cols_raw = df_raw.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("📊 원본 수치형 컬럼 개수:", len(numeric_cols_raw))
numeric_cols_raw


📊 원본 수치형 컬럼 개수: 7


['Unnamed: 0', '일자', '계획시간', '예상시간', '출발시간', '도착시간', '지연_분']

In [2]:
import numpy as np

df_time = df_raw.copy()

# 출발시간 정제
df_time["출발시간_clean"] = (
    df_time["출발시간"]
    .astype("Int64")
    .astype(str)
    .str.zfill(4)
)
df_time.loc[df_time["출발시간_clean"] == "<NA>", "출발시간_clean"] = np.nan

# datetime 생성
df_time["departure_datetime"] = pd.to_datetime(
    df_time["일자"].astype(str) + df_time["출발시간_clean"],
    format="%Y%m%d%H%M",
    errors="coerce"
)

df_time = df_time[df_time["departure_datetime"].notna()].copy()

# 시간 파생
df_time["dep_hour"] = df_time["departure_datetime"].dt.hour
df_time["dep_min"] = df_time["departure_datetime"].dt.minute
df_time["dep_weekday"] = df_time["departure_datetime"].dt.weekday
df_time["is_weekend"] = df_time["dep_weekday"].isin([5, 6]).astype(int)


In [3]:
numeric_compare = pd.DataFrame({
    "구분": ["기존 수치형 컬럼", "시간 파생 후 수치형 컬럼"],
    "컬럼 수": [len(numeric_cols_raw), 4],
    "컬럼 목록": [
        ", ".join(numeric_cols_raw),
        "dep_hour, dep_min, dep_weekday, is_weekend"
    ]
})

numeric_compare


,구분,컬럼 수,컬럼 목록
0,기존 수치형 컬럼,7,"Unnamed: 0, 일자, 계획시간, 예상시간, 출발시간, 도착시간, 지연_분"
1,시간 파생 후 수치형 컬럼,4,"dep_hour, dep_min, dep_weekday, is_weekend"


In [4]:
categorical_cols_raw = df_raw.select_dtypes(include=["object"]).columns.tolist()

print("📦 원본 범주형 컬럼 개수:", len(categorical_cols_raw))
categorical_cols_raw


📦 원본 범주형 컬럼 개수: 9


['출발/도착', '공항명', '항공사', '편명', '도착지', '구분', '상태', '지연원인', '출발지']

In [5]:
categorical_cols_selected = [
    "항공사",
    "출발지",
    "arrival_code",
    "flight_type"
]


In [6]:
categorical_compare = pd.DataFrame({
    "구분": ["기존 범주형 컬럼", "최종 선택 범주형 컬럼"],
    "컬럼 수": [len(categorical_cols_raw), len(categorical_cols_selected)],
    "컬럼 목록": [
        ", ".join(categorical_cols_raw),
        ", ".join(categorical_cols_selected)
    ]
})

categorical_compare


,구분,컬럼 수,컬럼 목록
0,기존 범주형 컬럼,9,"출발/도착, 공항명, 항공사, 편명, 도착지, 구분, 상태, 지연원인, 출발지"
1,최종 선택 범주형 컬럼,4,"항공사, 출발지, arrival_code, flight_type"


In [7]:
# =========================
# 📊 수치형 변수 ↔ 지연시간 상관관계
# =========================

target_col = "지연_분"

numeric_cols_final = [
    "dep_hour",
    "dep_min",
    "dep_weekday",
    "is_weekend"
]

df_num_corr = df_time[df_time[target_col].notna()].copy()

numeric_corr_table = (
    df_num_corr[numeric_cols_final + [target_col]]
    .corr(method="pearson")[[target_col]]
    .rename(columns={target_col: "지연_분과의 상관계수"})
    .sort_values(by="지연_분과의 상관계수", ascending=False)
)

numeric_corr_table


,지연_분
지연_분,1.000000
dep_hour,0.136242
dep_min,0.014193
dep_weekday,0.001631
is_weekend,-0.012311


In [9]:
# 전처리 후 (arrival_code 포함)
df_processed = pd.read_csv("new_flight_analysis_summary.csv")

In [11]:
import numpy as np

def eta_squared(df, cat_col, target_col):
    """
    범주형 변수(cat_col)가
    연속형 타깃(target_col)을
    얼마나 설명하는지 (η²) 계산
    """
    df = df[[cat_col, target_col]].dropna()

    grand_mean = df[target_col].mean()

    # 그룹 평균
    group_means = df.groupby(cat_col)[target_col].mean()
    group_counts = df.groupby(cat_col)[target_col].count()

    # Between-group SS
    ss_between = ((group_means - grand_mean) ** 2 * group_counts).sum()

    # Total SS
    ss_total = ((df[target_col] - grand_mean) ** 2).sum()

    return ss_between / ss_total


In [12]:
target_col = "지연_분"

categorical_cols = [
    "항공사",
    "출발지",
    "arrival_code",
    "flight_type"
]

eta_results = []

for col in categorical_cols:
    eta = eta_squared(df_processed, col, target_col)
    eta_results.append({
        "범주형 변수": col,
        "η² (설명력)": round(eta, 4)
    })

eta_df = pd.DataFrame(eta_results).sort_values(
    by="η² (설명력)", ascending=False
)

eta_df


,범주형 변수,η² (설명력)
1,출발지,0.0462
2,arrival_code,0.0367
0,항공사,0.0248
3,flight_type,0.0171
